In [1]:
from openai import Client
from pprint import pprint
import pandas as pd
from llm_sandbox import SandboxSession
import json
import sqlite3
import os


In [2]:
client = Client()
db = sqlite3.connect("database.sqlite")


In [10]:
tools = [
  {
      "type": "function",
      "function": {
          "name": "run_sql_and_view_table",
          "parameters": {
              "type": "object",
              "properties": {
                  "sql_query": {"type": "string"}
              },
          },
      },
  },
  {
    "type": "function",
    "function": {
      "name": "run_sql_and_save_to_parquet",
      "parameters": {
        "type": "object",
        "properties": {
          "sql_query": {"type": "string"},
          "file_name": {"type": "string"}
        },
      },
    },
  },
  {
      "type": "function",
      "function": {
          "name": "run_python",
          "parameters": {
              "type": "object",
              "properties": {
                  "python_code": {"type": "string"}
              },
          },
      },
  },
]

In [11]:
schema_description = """### Схема базы данных

#### Таблица: `users` (Пользователи)
- **id**: INTEGER PRIMARY KEY  
  Уникальный идентификатор пользователя.
- **name**: TEXT  
  Полное имя пользователя.
- **email**: TEXT  
  Электронная почта пользователя. Уникальная для каждого пользователя.
- **phone**: TEXT  
  Телефонный номер пользователя.
- **address**: TEXT  
  Адрес пользователя.

---

#### Таблица: `employees` (Работники)
- **id**: INTEGER PRIMARY KEY  
  Уникальный идентификатор работника.
- **name**: TEXT  
  Полное имя работника.
- **position**: TEXT  
  Должность работника.
- **salary**: REAL  
  Зарплата работника.
- **hire_date**: TEXT  
  Дата приема на работу.

---

#### Таблица: `products` (Товары)
- **id**: INTEGER PRIMARY KEY  
  Уникальный идентификатор товара.
- **name**: TEXT  
  Название товара.
- **price**: REAL  
  Цена товара.
- **stock**: INTEGER  
  Количество товара на складе.

---

#### Таблица: `orders` (Заказы)
- **id**: INTEGER PRIMARY KEY  
  Уникальный идентификатор заказа.
- **user_id**: INTEGER  
  Идентификатор пользователя, сделавшего заказ (ссылается на `users.id`).
- **product_id**: INTEGER  
  Идентификатор товара в заказе (ссылается на `products.id`).
- **quantity**: INTEGER  
  Количество единиц товара в заказе.
- **order_date**: TEXT  
  Дата оформления заказа.

---

### Взаимосвязи между таблицами
1. **Таблица `orders`**:
   - Поле `user_id` связано с таблицей `users` через `users.id`.
   - Поле `product_id` связано с таблицей `products` через `products.id`.

### Примерные запросы для аналитиков
1. **Список всех заказов с данными о пользователях и товарах:**
   ```sql
   SELECT orders.id, users.name AS user_name, products.name AS product_name, orders.quantity, orders.order_date
   FROM orders
   JOIN users ON orders.user_id = users.id
   JOIN products ON orders.product_id = products.id;
   ```

2. **Сумма продаж по каждому товару:**
   ```sql
   SELECT products.name, SUM(orders.quantity * products.price) AS total_revenue
   FROM orders
   JOIN products ON orders.product_id = products.id
   GROUP BY products.id;
   ```

3. **Список работников с зарплатой выше 50,000:**
   ```sql
   SELECT * FROM employees WHERE salary > 50000;
   ```

4. **Складские остатки товаров:**
   ```sql
   SELECT name, stock FROM products WHERE stock > 0;
   ```"""

In [12]:
def sql2pandas(sql_query: str) -> pd.DataFrame:
    global db
    cursor = db.cursor()
    cursor.execute(sql_query)
    result = cursor.fetchall()
    return pd.DataFrame(result)


def sql2parquet(sql_query: str, file_name: str) -> None:
    df = sql2pandas(sql_query)
    df.to_parquet(file_name)

def sql2str(sql_query: str) -> str:
    """Возвращает строку с первыми 4 строками таблицы. Так же отчищает от лишних символов итп, что бы тратить меньше токенов"""
    df = sql2pandas(sql_query)
    df = df.iloc[:4]
    for column in df.columns:
        df[column] = df[column].astype(str)
        df[column] = df[column].apply(lambda x: x[:200] + "..." if len(x) > 200 else x)
    return df.to_string()
    

In [29]:
SYSTEM_MESSAGE: dict = {"role": "system", "content": "You are a helpful assistant that can run SQL commands. \n Database schema: \n" + schema_description}
history: list[dict] = [SYSTEM_MESSAGE]
need_user_intent: bool = True
while True:
    if need_user_intent:
        user_input: str = input()
        if user_input == "exit":
            break
        history.append({"role": "user", "content": user_input})
    completion = client.chat.completions.create(
        model="gpt-4o",
        messages=history,
        tools=tools,
    )
    llm_answer: str = completion.choices[0].message.content
    history.append({"role": "assistant", "content": llm_answer})
    if completion.choices[0].message.tool_calls:
        tool_call: dict = completion.choices[0].message.tool_calls[0]
        tool_name: str = tool_call.function.name
        tool_args = tool_call.function.arguments
        if tool_name == "run_sql_and_view_table":
            sql_query: str = json.loads(tool_args)["sql_query"]
            result: str = sql2str(sql_query)
            history.append({"role": "tool", "content": result})
        if tool_name == "run_sql_and_save_to_parquet":
            sql_query: str = json.loads(tool_args)["sql_query"]
            file_name: str = json.loads(tool_args)["file_name"]
            sql2parquet(sql_query, file_name)
            history.append({"role": "tool", "content": f"File {file_name} saved"})
        if tool_name == "run_python":
            python_code: str = json.loads(tool_args)["python_code"]
            result = exec(python_code)
            history.append({"role": "tool", "content": f"Python code executed with result: {result}"})
    else:
        need_user_intent = True
        print("LLM says: ", completion.choices[0].message.content)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid value for 'content': expected a string, got null.", 'type': 'invalid_request_error', 'param': 'messages.[2].content', 'code': None}}

In [30]:
history

[{'role': 'system',
  'content': 'You are a helpful assistant that can run SQL commands. \n Database schema: \n### Схема базы данных\n\n#### Таблица: `users` (Пользователи)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор пользователя.\n- **name**: TEXT  \n  Полное имя пользователя.\n- **email**: TEXT  \n  Электронная почта пользователя. Уникальная для каждого пользователя.\n- **phone**: TEXT  \n  Телефонный номер пользователя.\n- **address**: TEXT  \n  Адрес пользователя.\n\n---\n\n#### Таблица: `employees` (Работники)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор работника.\n- **name**: TEXT  \n  Полное имя работника.\n- **position**: TEXT  \n  Должность работника.\n- **salary**: REAL  \n  Зарплата работника.\n- **hire_date**: TEXT  \n  Дата приема на работу.\n\n---\n\n#### Таблица: `products` (Товары)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор товара.\n- **name**: TEXT  \n  Название товара.\n- **price**: REAL  \n  Цена товара.\n- **stock*

In [26]:
SYSTEM_MESSAGE: dict = {"role": "system", "content": "You are a helpful assistant that can run SQL commands. \n Database schema: \n" + schema_description}
SYSTEM_MESSAGE

({'role': 'system',
  'content': 'You are a helpful assistant that can run SQL commands. \n Database schema: \n### Схема базы данных\n\n#### Таблица: `users` (Пользователи)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор пользователя.\n- **name**: TEXT  \n  Полное имя пользователя.\n- **email**: TEXT  \n  Электронная почта пользователя. Уникальная для каждого пользователя.\n- **phone**: TEXT  \n  Телефонный номер пользователя.\n- **address**: TEXT  \n  Адрес пользователя.\n\n---\n\n#### Таблица: `employees` (Работники)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор работника.\n- **name**: TEXT  \n  Полное имя работника.\n- **position**: TEXT  \n  Должность работника.\n- **salary**: REAL  \n  Зарплата работника.\n- **hire_date**: TEXT  \n  Дата приема на работу.\n\n---\n\n#### Таблица: `products` (Товары)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор товара.\n- **name**: TEXT  \n  Название товара.\n- **price**: REAL  \n  Цена товара.\n- **stock*

In [20]:
SYSTEM_MESSAGE

({'role': 'system',
  'content': 'You are a helpful assistant that can run SQL commands. \n Database schema: \n### Схема базы данных\n\n#### Таблица: `users` (Пользователи)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор пользователя.\n- **name**: TEXT  \n  Полное имя пользователя.\n- **email**: TEXT  \n  Электронная почта пользователя. Уникальная для каждого пользователя.\n- **phone**: TEXT  \n  Телефонный номер пользователя.\n- **address**: TEXT  \n  Адрес пользователя.\n\n---\n\n#### Таблица: `employees` (Работники)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор работника.\n- **name**: TEXT  \n  Полное имя работника.\n- **position**: TEXT  \n  Должность работника.\n- **salary**: REAL  \n  Зарплата работника.\n- **hire_date**: TEXT  \n  Дата приема на работу.\n\n---\n\n#### Таблица: `products` (Товары)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор товара.\n- **name**: TEXT  \n  Название товара.\n- **price**: REAL  \n  Цена товара.\n- **stock*

In [21]:
history

[({'role': 'system',
   'content': 'You are a helpful assistant that can run SQL commands. \n Database schema: \n### Схема базы данных\n\n#### Таблица: `users` (Пользователи)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор пользователя.\n- **name**: TEXT  \n  Полное имя пользователя.\n- **email**: TEXT  \n  Электронная почта пользователя. Уникальная для каждого пользователя.\n- **phone**: TEXT  \n  Телефонный номер пользователя.\n- **address**: TEXT  \n  Адрес пользователя.\n\n---\n\n#### Таблица: `employees` (Работники)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор работника.\n- **name**: TEXT  \n  Полное имя работника.\n- **position**: TEXT  \n  Должность работника.\n- **salary**: REAL  \n  Зарплата работника.\n- **hire_date**: TEXT  \n  Дата приема на работу.\n\n---\n\n#### Таблица: `products` (Товары)\n- **id**: INTEGER PRIMARY KEY  \n  Уникальный идентификатор товара.\n- **name**: TEXT  \n  Название товара.\n- **price**: REAL  \n  Цена товара.\n- **stoc

In [9]:
exec("print(1)")

1


In [18]:
pprint(completion.choices[0].to_dict())

{'finish_reason': 'tool_calls',
 'index': 0,
 'logprobs': None,
 'message': {'content': None,
             'refusal': None,
             'role': 'assistant',
             'tool_calls': [{'function': {'arguments': '{"sql_query":"SELECT '
                                                       'temperature, '
                                                       'weather_description '
                                                       'FROM city_weather '
                                                       'WHERE city_name = '
                                                       '\'Paris\'"}',
                                          'name': 'run_sql_command'},
                             'id': 'call_LPlMsPZc1rViIpymV97ohAN9',
                             'type': 'function'}]}}


In [12]:
llm_code = """
import numpy as np\n\n# Данные о суммах заказов\\norder_values = np.array([2837.8, 3669.48, 761.91, 3700.4])\\n\\n# Рассчитываем медиану, 90-й и 99-й перцентили\\nmedian = np.median(order_values)\\npct_90 = np.percentile(order_values, 90)\\npct_99 = np.percentile(order_values, 99)\\n\\n# Вернуть значения\\noutput = {\'median\': median, \'percentile_90\': pct_90, \'percentile_99\': pct_99}\\noutput"}
"""


In [9]:
from llm_sandbox import SandboxSession

# Create a new sandbox session}
with SandboxSession(lang="python") as session:
    result = session.run("print('Hello, World!')")
    print(result)


ConsoleOutput(text=Hello, World!
)


In [10]:
with SandboxSession(lang="python") as session:
    # result = session.run("""import numpy as np\n)
    print(result)

NameError: name 'llm_code' is not defined

In [11]:
llm_code

NameError: name 'llm_code' is not defined

In [18]:
r = eval('import numpy as np\nprint(np.random.randint(1, 100))')

SyntaxError: invalid syntax (<string>, line 1)

In [7]:
with SandboxSession(lang="python", dockerfile=os.path.abspath('Dockerfile')) as session:
    result = session.run("""print(1 + 2)""")
    print(result.text)
    session.run("""x = 1""")
    session.run("""print(x)""")
    print(result.text)

BuildError: The command '/bin/sh -c pip install -r /sandbox_reqs.txt' returned a non-zero code: 1

In [3]:
session.path

NameError: name 'session' is not defined

In [13]:
with SandboxSession(lang="python", dockerfile=os.path.abspath('Dockerfile')) as session:
    session.copy_to_runtime(src='database.sqlite', dest='/')
    res = session.run("""import os; print(os.getcwd()); print(os.listdir())""")
    print(res.text)


/
['tmp', 'bin', 'srv', 'proc', 'home', 'root', 'run', 'lib', 'etc', 'dev', 'mnt', 'sbin', 'opt', 'usr', 'sys', 'var', 'boot', 'media', 'database.sqlite', '.dockerenv', 'sandbox_reqs.txt']

